# Empirical Final Project

### SOCI-20559 Spatial Regression Analysis

#### Empirical Assignment 

#### Polina Rozhkova

#### 5/26/2023

The objective of this research is to assess the spatial relationship between the accessibility of opioid treatment programs/opioid treatment medication clinics and opioid overdose fatalities across Chicago’s 77 Community Areas. My key explanatory variable is the concentration of licensed buprenorphine providers in a community area. Additional factors that vary by environment and may affect the rate of fatal overdoses in given area include demographic data (race, percent of population living in poverty, median income, unemployment rate, percent of population that is uninsured) and accessibility to drugs such as narcan which I measure using the concentration of pharmacies in a community area. The dependent variable will be the overdose mortality rate per 100 (in an attempt to standardize across community areas that vary significantly in population size). 

I will start by assessing a simple OLS regression and running tests for spatial effects before moving to a regression with spatial dependence. Finally, I will explore the use of spatial regimes to address spatial heterogeneity. Related studies employ geographically weighted regression (when the dependent variable is mortality rate) and other studies have used: logistic regressions and Poisson regression model (where the dependent variable is binary and is composed of fatal and non-fatal overdoses). Ultimately, this is the main limitation of my project and while working with areal units was my best option given the data I'm working with, it greately reduced the degrees of freedom specifically when attempting to employ spatial regimes. 

*Background*

Intervention and public discourse around opioid-use disorder (and related substance use disorder) have led to the creation of the “opioid epidemic”, a public health emergency devastating every state in the nation, targeting predominantly white lower income Americans living in rural areas (Griffith et al. 2018, 843-844). We know however that heroin addiction and related substance-use disorders are far from a new phenomenon. As of late, it’s become evident (or more widely accepted) that prescription opioid overuse and abuse has impacted and continues to devastate individuals across racial and socioeconomic groups. Treatment options and care have been primarily targeted at white communities and have ignored the detrimental effects on Black people thereby reinforcing the racial inequities. 
Different groups require different and nuanced forms of treatment and support to live with addiction. Substance use disorder like most mental disorders can be the result of trauma, environment, negative or unlucky circumstances, and social pressures. Because addiction is so stratified, what may work for one individual might not work for another individual living in a different environment with less resources. One category of individuals with opioid use disorder may have greater access to prescription opiates, another category may be more likely to access “street drug” alternatives, and another relevant group is composed of individuals who are actively in treatment for opioid use disorder who then relapse due to decreased tolerance often resulting in death.


In [1]:
import numpy as np
import pandas as pd
import os
os.environ['USE_PYGEOS'] = '0'
import geopandas as gpd
import libpysal
import esda
import spreg
import time

In [2]:
spreg.__version__

'1.3.2'

In [3]:
pd.set_option('display.max_columns', None)

In [4]:
path = r'/Users/polinarozhkova/Desktop/GitHub/moud_access/data_final'

In [5]:
area_shp = gpd.read_file(os.path.join(path, 'Boundaries - Community Areas (current)',
                        'geo_export_122237a7-de0c-463d-b81f-e6d53bf2e92a.shp'))
od_df = pd.read_csv(os.path.join(path, 'chicago_overdose.csv'))
dem_df = pd.read_csv(os.path.join(path, 'chicago_demograph.csv'))

### Load and Explore Data

**od_df**: Opioid related mortality records are available through the Cook County Medical Examiner’s data portal including, individual characteristics, as well as the community area and coordinates of the deceased individual’s residence. I compiled buprenorphine provider location data from the Substance Abuse and Mental Health Services Administration and pharmacy location data from the Chicago city data portal. 

**dem_df**: The Heartland Alliance gathers demographic data from the American Community Survey on community health and economics which include race, population estimates for each community area, median household income, the percentage of individuals living in poverty, unemployment rate, and the percentage of uninsured population. While all these variables are related, and will likely present multicollinearity, they indicate slightly different characteristics that might be at play in different communities.  

Community areas most impacted by opioid related overdose deaths are Austin, East Garfield, West Garfield, Humboldt Park, and North Lawndale. Though steadily increasing between 2019 and 2021, there seem to be no major changes in the areas that appear to be most heavily impacted (community areas with the highest rates of opioid related mortality in 2019 remain high through 2020 and 2021). For the project, I plan to use the 2021 opioid mortality data and demographic data from 2020—the findings for these years might not be generalizable but could be compared to cross sections from past years or future data.  

In [6]:
#area_shp.head()

Fatal overdoses, buprenorphone providers, and pharmacy locations were joined to the community area shapefile and aggregated.

In [7]:
#od_df.head()

In [8]:
od_df.describe()

,area_num_1,region,shape_area,shape_len,od_2019,od_2020,od_2021,bupren_area,pharmacy_area
count,77.000000,77.000000,7.700000e+01,77.000000,77.000000,77.000000,77.000000,77.000000,77.000000
mean,39.000000,3.896104,8.361453e+07,44397.606964,10.909091,15.857143,17.597403,7.181818,5.441558
std,22.371857,2.010397,5.494626e+07,20090.463816,13.945554,20.064839,22.436612,17.614832,5.608953
min,1.000000,1.000000,1.691396e+07,18137.944253,0.000000,0.000000,0.000000,0.000000,0.000000
25%,20.000000,2.000000,4.976964e+07,31948.598840,3.000000,4.000000,4.000000,0.000000,2.000000
50%,39.000000,4.000000,7.963575e+07,43229.372704,6.000000,9.000000,8.000000,2.000000,3.000000
75%,58.000000,6.000000,9.885317e+07,49478.427771,14.000000,18.000000,26.000000,5.000000,7.000000
max,77.000000,7.000000,3.718356e+08,173625.984660,88.000000,106.000000,138.000000,131.000000,25.000000


In [9]:
#dem_df.head()

In [10]:
dem_df.iloc[1:].describe() 

,% in Poverty,% in Extreme Poverty,Child Poverty Rate,% Female,% Male,% Black,% Hispanic,% White,% Aged 0-4,% Aged 5-17,% Aged 18-24,% Aged 25-64,% Aged 65+,Total Pop.,Change in Pop. (2018 to 2019),Overall Unemp.,Unemp. 16-19 yr. olds,Unemp. 20-24 yr. olds,% no HS Diploma,% HS or GED,Some College,% Associates Deg.,% Bachelors Deg.,Homeownership Rate,Rent Burden,SNAP Enrollment Rate,% Cash Assist.,Uninsured Rate,Avg. Med. Household Income (2020 Dollars)
count,77.000000,77.000000,77.000000,77.000000,77.000000,77.000000,77.000000,77.000000,77.000000,77.000000,77.000000,77.000000,77.000000,77.000000,77.000000,77.000000,77.000000,77.000000,77.000000,77.000000,77.000000,77.000000,77.000000,77.000000,77.000000,77.000000,77.000000,77.000000,77.000000
mean,19.070130,8.907792,-0.406494,52.272727,47.727273,36.907792,26.449351,27.890909,6.238961,15.236364,9.550649,54.898701,14.070130,35060.361039,0.001299,10.536364,33.006494,19.274026,15.242857,24.890909,19.471429,6.155844,34.246753,48.470130,49.896104,21.287013,3.427273,10.607792,60061.219481
std,10.982032,6.205023,2.910523,3.330388,3.330388,38.575052,27.459035,26.347292,1.711813,4.695572,3.251705,6.716936,4.776782,23114.726472,0.025616,6.674084,21.183430,12.692511,9.146576,10.138828,7.052936,1.924940,22.080526,19.223882,11.360220,14.523849,1.806686,4.823238,24285.115980
min,3.300000,1.700000,-6.100000,42.400000,38.700000,0.400000,0.000000,0.800000,0.900000,2.600000,2.100000,40.500000,5.900000,2158.200000,-0.100000,0.700000,0.000000,0.000000,1.800000,3.300000,6.800000,1.800000,6.700000,8.800000,24.500000,1.500000,0.600000,1.500000,25700.000000
25%,10.900000,4.300000,-2.200000,50.100000,45.400000,3.000000,5.400000,4.400000,5.400000,11.600000,7.700000,50.500000,10.600000,18337.800000,0.000000,5.100000,18.500000,9.000000,8.100000,18.900000,14.400000,4.900000,16.300000,35.200000,41.600000,9.200000,2.100000,8.200000,42576.200000
50%,15.700000,6.700000,-0.300000,51.700000,48.300000,13.300000,13.200000,14.700000,6.300000,16.000000,9.200000,52.800000,13.200000,29489.900000,0.000000,8.500000,29.400000,15.300000,13.600000,25.600000,19.300000,6.100000,28.600000,45.300000,50.600000,17.300000,3.300000,10.400000,52596.600000
75%,25.500000,12.100000,0.500000,54.600000,49.900000,82.600000,45.900000,48.800000,7.200000,18.400000,10.900000,58.500000,16.600000,46747.800000,0.000000,16.100000,48.500000,27.800000,20.800000,32.100000,25.100000,7.600000,43.600000,64.500000,58.700000,33.000000,4.600000,12.500000,74054.100000
max,47.600000,34.800000,12.300000,61.300000,57.600000,96.500000,91.000000,82.700000,11.400000,24.000000,24.600000,75.900000,28.500000,101392.000000,0.100000,29.700000,100.000000,51.600000,41.000000,42.200000,42.700000,10.200000,85.300000,91.700000,72.500000,61.600000,9.100000,24.000000,128699.800000


In [11]:
dem_df2 = dem_df[['community', 'Total Pop.', '% in Poverty',
                        '% in Extreme Poverty', '% Female',
                        '% Black', '% Hispanic', '% White', '% Aged 0-4',
                        '% Aged 5-17', '% Aged 18-24',
                        '% Aged 25-64', '% Aged 65+',
                        'Overall Unemp.', 'Unemp. 20-24 yr. olds',
                        '% no HS Diploma', '% HS or GED',
                        '% Bachelors Deg.', 'Homeownership Rate',
                        'SNAP Enrollment Rate', '% Cash Assist.',
                        'Uninsured Rate',
                        'Avg. Med. Household Income (2020 Dollars)']]

In [12]:
od_df2 = od_df.merge(dem_df2, how='inner', indicator=True) #drop Chicago overall

In [13]:
od_df2.shape

(77, 34)

In [14]:
#od_df2.columns

Standardizing the dependent variable, number of fatal overdoses in a given area, and the key independent variables, the concentration of buprenorphine and pharmacies in a given community area.

Overdose rate per 100 (minimum pop. is 2158)

In [15]:
def spat_intensive(df, var1, var2, var3, var4, var5):
    df[var1]  = (df[var1]/df['Total Pop.'])*100
    df[var2]  = (df[var2]/df['Total Pop.'])*100
    df[var3]  = (df[var3]/df['Total Pop.'])*100
    df[var4]  = (df[var4]/df['Total Pop.'])*100
    df[var5]  = (df[var5]/df['Total Pop.'])*100
    return df

od_df2 = spat_intensive(od_df2, 'od_2019', 'od_2020', 'od_2021', 'bupren_area', 'pharmacy_area')
#od_df2

In [16]:
wq = libpysal.io.open(os.path.join(path, "Boundaries - Community Areas (current)/q_order1.gal")).read()
wq.transform = 'r'
# wq.weights

In [17]:
wq.n
wq.weights['77']

[0.25, 0.25, 0.25, 0.25]

### Model Specification

I have three dependent variables, the overdose rate in 2019, the overdose rate in 2020, and the overdose rate in 2021. I will start with 2019. 

In [18]:
y_name1 = 'od_2019'
y_name2 = 'od_2020'
y_name3 = 'od_2021'

In [19]:
x_names1 = ['bupren_area', 'pharmacy_area']
x_names2 = ['bupren_area', 'pharmacy_area', 
            '% in Poverty',
            '% Female',
            '% Black',
            '% Hispanic',
            '% White', 
            '% Aged 65+',
            'Overall Unemp.',
            'Unemp. 20-24 yr. olds',
            '% no HS Diploma',
            '% HS or GED',
            '% Bachelors Deg.',
            'Uninsured Rate',
            #'Avg. Med. Household Income (2020 Dollars)'
           ]

In [20]:
ds_name = 'od_df2'
w_name = 'q_order1'

In [21]:
y1 = np.array(od_df2[y_name1])
y2 = np.array(od_df2[y_name2])
y3 = np.array(od_df2[y_name3])
y1.shape

(77,)

In [22]:
x1 = np.array(od_df2[x_names1])
x1.shape

(77, 2)

In [23]:
x2 = np.array(od_df2[x_names2])
x2.shape

(77, 14)

### OLS Regression

### LM Diagnostics 

In [24]:
ols1b = spreg.OLS(y3, x1, w=wq, name_y=y_name3, name_x=x_names1, name_ds=ds_name,
                 spat_diag=True, moran=True, name_w=w_name)
print(ols1b.summary)

REGRESSION
----------
SUMMARY OF OUTPUT: ORDINARY LEAST SQUARES
-----------------------------------------
Data set            :      od_df2
Weights matrix      :    q_order1
Dependent Variable  :     od_2021                Number of Observations:          77
Mean dependent var  :      0.0574                Number of Variables   :           3
S.D. dependent var  :      0.0830                Degrees of Freedom    :          74
R-squared           :      0.0660
Adjusted R-squared  :      0.0407
Sum squared residual:       0.489                F-statistic           :      2.6143
Sigma-square        :       0.007                Prob(F-statistic)     :     0.07997
S.E. of regression  :       0.081                Log likelihood        :      85.483
Sigma-square ML     :       0.006                Akaike info criterion :    -164.966
S.E of regression ML:      0.0797                Schwarz criterion     :    -157.934

-----------------------------------------------------------------------------

In [25]:
ols2b = spreg.OLS(y3, x2, w=wq, name_y=y_name3, name_x=x_names2, name_ds=ds_name,
                 spat_diag=True, moran=True, name_w=w_name)
print(ols2b.summary)

REGRESSION
----------
SUMMARY OF OUTPUT: ORDINARY LEAST SQUARES
-----------------------------------------
Data set            :      od_df2
Weights matrix      :    q_order1
Dependent Variable  :     od_2021                Number of Observations:          77
Mean dependent var  :      0.0574                Number of Variables   :          15
S.D. dependent var  :      0.0830                Degrees of Freedom    :          62
R-squared           :      0.6431
Adjusted R-squared  :      0.5626
Sum squared residual:       0.187                F-statistic           :      7.9813
Sigma-square        :       0.003                Prob(F-statistic)     :   2.456e-09
S.E. of regression  :       0.055                Log likelihood        :     122.525
Sigma-square ML     :       0.002                Akaike info criterion :    -215.050
S.E of regression ML:      0.0493                Schwarz criterion     :    -179.893

-----------------------------------------------------------------------------

There appears to be strong evidence of spatial misspecification based on the Moran's I statistic. Both the LM-error and LM-lag statistics reject the null in favor of spatial autocorrelation so it makes sense to rely on the robust tests. With this model, neither robust statistic is statistically significant but because the robust LM-lag statistic is closer to signficiance, I will proceed with estimating a spatial lag model. There's very little additional evidence that this would be useful. 

As anticipated, the multicollinearity is high and I will experiment with removing some of the explanatory variables with the tradeoff of adjusted R-squared.

In [26]:
t0 = time.time()
lag1 = spreg.ML_Lag(y3,x2,w=wq,name_y=y_name3,name_x=x_names2,
                     name_w=w_name,name_ds=ds_name)
t1 = time.time()
print(t1-t0)
print(lag1.summary)

0.02537083625793457
REGRESSION
----------
SUMMARY OF OUTPUT: MAXIMUM LIKELIHOOD SPATIAL LAG (METHOD = FULL)
-----------------------------------------------------------------
Data set            :      od_df2
Weights matrix      :    q_order1
Dependent Variable  :     od_2021                Number of Observations:          77
Mean dependent var  :      0.0574                Number of Variables   :          16
S.D. dependent var  :      0.0830                Degrees of Freedom    :          61
Pseudo R-squared    :      0.7148
Spatial Pseudo R-squared:  0.5977
Sigma-square ML     :       0.002                Log likelihood        :     128.467
S.E of regression   :       0.044                Akaike info criterion :    -224.934
                                                 Schwarz criterion     :    -187.433

------------------------------------------------------------------------------------
            Variable     Coefficient       Std.Error     z-Statistic     Probability
---------

/Users/polinarozhkova/anaconda3/lib/python3.10/site-packages/scipy/optimize/_minimize.py:892: RuntimeWarning: Method 'bounded' does not support relative tolerance in x; defaulting to absolute tolerance.
  warn("Method 'bounded' does not support relative tolerance in x; "


The estimated spatial autoregressive coefficient is 0.529 and appears to be highly statistically signficant and the explanatory variables that were statistically significant in the OLS model maintain signficiance (or become more significant) while maintaining cofficient sign. The magnitude of the effects of these explanatory variables are  small relative to the estimated coefficient on *pharmacy_area*. The actual interpretation of this coefficient doesn't make much practical sense but suggests that an increase in the concentration of pharmacies (number of pharmacies/total population of community area) per 100 people is estimated to increase the rate of fatal overdose in a community holding everything else constant. Finally, the overall AIC of the spatial lag model is smaller (-224.934) than it is in the OLS model (-215.050) which can suggest an improvement in the fit of the model. I won't hold onto that possibility too dearly. 

## Spatial Regimes

For the regimes variable:

In [27]:
regimes = od_df2['region'].tolist()
type(regimes)

list

### Spatially Lagged Explanatory Variables

In [28]:
WX_1 = libpysal.weights.lag_spatial(wq,x1)
xdur_1 = np.hstack((x1,WX_1))
xdur_1[0:3,]

array([[0.02797203, 0.00932401, 0.01193153, 0.01987024],
       [0.01434103, 0.02868206, 0.02289167, 0.01245072],
       [0.04521818, 0.04521818, 0.01364934, 0.01392206]])

In [29]:
WX_2 = libpysal.weights.lag_spatial(wq,x2)
xdur_2 = np.hstack((x2,WX_2))
#xdur_2[0:3,]

In [30]:
wnames_1 = []
for j in x_names1:
    wnames_1.append("W_" + j)
dur_names_1 = x_names1 + wnames_1
dur_names_1

['bupren_area', 'pharmacy_area', 'W_bupren_area', 'W_pharmacy_area']

In [31]:
wnames_2 = []
for j in x_names2:
    wnames_2.append("W_" + j)
dur_names_2 = x_names2 + wnames_2
#dur_names_2

### Spatially Lagged Dependent Variable

We will also need to explicitly include the lagged dependent variable as an endogenous variable.

In [32]:
Wy_1 = libpysal.weights.lag_spatial(wq,y1).reshape(-1,1)
yend_name_1 = ["Wy_1"]
Wy_1.shape

(77, 1)

In [33]:
Wy_2 = libpysal.weights.lag_spatial(wq,y2).reshape(-1,1)
yend_name_2 = ["Wy_2"]
Wy_2.shape

(77, 1)

In [34]:
Wy_3 = libpysal.weights.lag_spatial(wq,y3).reshape(-1,1)
yend_name_3 = ["Wy_3"]
Wy_3.shape

(77, 1)

### Additional Instruments


In [35]:
W2X_1 = libpysal.weights.lag_spatial(wq,WX_1)
w2names_1 = []
for j in x_names1:
    w2names_1.append("W2_" + j)
w2names_1

['W2_bupren_area', 'W2_pharmacy_area']

In [36]:
W2X_2 = libpysal.weights.lag_spatial(wq,WX_2)
w2names_2 = []
for j in x_names2:
    w2names_2.append("W2_" + j)
#w2names_2

In [37]:
wwwx = libpysal.weights.lag_spatial(wq,W2X_2)
W3X = np.hstack((W2X_2,wwwx))
w3names = []
for j in x_names2:
    w3names.append("W3_" + j)
inst3names = w2names_2 + w3names
#inst3names

In [38]:
t0 = time.time()
reg_1_regime = spreg.OLS_Regimes(y3, x1, regimes, w=wq, spat_diag=True,
                        name_y=y_name3, name_x=x_names1, #name_regimes=rvar,
                        name_w=w_name,name_ds=ds_name)
t1 = time.time()
print(t1-t0)
print(reg_1_regime.summary)

/Users/polinarozhkova/anaconda3/lib/python3.10/site-packages/spreg/utils.py:153: RuntimeWarning: divide by zero encountered in scalar divide
  self._cache["sig2n_k"] = self.utu / (self.n - self.k)
/Users/polinarozhkova/anaconda3/lib/python3.10/site-packages/spreg/diagnostics.py:294: RuntimeWarning: invalid value encountered in scalar divide
  ar2_result = 1 - (1 - r2(reg)) * (n - 1) / (n - k)
/Users/polinarozhkova/anaconda3/lib/python3.10/site-packages/spreg/diagnostics.py:96: RuntimeWarning: divide by zero encountered in scalar divide
  fStat = (U / (k - 1)) / (Q / (n - k))


0.0648648738861084
REGRESSION
----------

SUMMARY OF OUTPUT: ORDINARY LEAST SQUARES ESTIMATION - REGIME 1
---------------------------------------------------------------
Data set            :      od_df2
Weights matrix      :    q_order1
Dependent Variable  :   1_od_2021                Number of Observations:          12
Mean dependent var  :      0.0450                Number of Variables   :           3
S.D. dependent var  :      0.0290                Degrees of Freedom    :           9
R-squared           :      0.0070
Adjusted R-squared  :     -0.2137
Sum squared residual:       0.009                F-statistic           :      0.0316
Sigma-square        :       0.001                Prob(F-statistic)     :       0.969
S.E. of regression  :       0.032                Log likelihood        :      26.004
Sigma-square ML     :       0.001                Akaike info criterion :     -46.008
S.E of regression ML:      0.0277                Schwarz criterion     :     -44.553

-------------

In [39]:
#t0 = time.time()
#reg_2_regime = spreg.OLS_Regimes(y3, x2, regimes, w=wq, spat_diag=True,
#                        name_y=y_name3, name_x=x_names2, #name_regimes=rvar,
#                        name_w=w_name,name_ds=ds_name)
#t1 = time.time()
#print(t1-t0)
#print(reg_2_regime.summary)

Attempting to include additional explanatory variables with spatial regimes results in major issues as there are not enough observations per regime and decreasing the number of groups does not help the problem of few observations. Further, this may mask heterogeneity that would actually provide useful and/or interesting information. 

My primary focus with this analysis was to capture and examine the effect of the density of buprenorphine providers on the fatal overdose rate. Based on the explanatory variables I've selected, the concentration of buprenorphone providers in a given community area does not appear to have a statistically significant effect on the rate of fatal overdoses in that area. However, the concentration of pharmacies in a given area is statistically signficiant in the model including all the explanatory variables and holding constant the effects of these variables, a higher rate of pharmacies per community area has a positive effect on the rate of fatal overdoses. This is in no way a generalizable finding but I decided to explore this a bit more by looking at what happens to the estimates on the other explanatory variables when I omit buprenorphine providers and maintain the variables that are statistically significant or close to significant in the previous OLS regression (concentration of pharmacies, the percent of Black residents, the percent of White reseidents, and the percent of residents with no HS diploma). 

In [40]:
x_names2 = [#'bupren_area', 
            'pharmacy_area', 
            '% in Poverty',
            '% Black',
            '% White', 
            '% no HS Diploma',
            'Uninsured Rate'
           ]

In [41]:
x2 = np.array(od_df2[x_names2])

In [42]:
t0 = time.time()
lag2 = spreg.ML_Lag(y3,x2,w=wq,name_y=y_name3,name_x=x_names2,
                     name_w=w_name,name_ds=ds_name)
t1 = time.time()
print(t1-t0)
print(lag2.summary)

0.016043663024902344
REGRESSION
----------
SUMMARY OF OUTPUT: MAXIMUM LIKELIHOOD SPATIAL LAG (METHOD = FULL)
-----------------------------------------------------------------
Data set            :      od_df2
Weights matrix      :    q_order1
Dependent Variable  :     od_2021                Number of Observations:          77
Mean dependent var  :      0.0574                Number of Variables   :           8
S.D. dependent var  :      0.0830                Degrees of Freedom    :          69
Pseudo R-squared    :      0.6653
Spatial Pseudo R-squared:  0.5921
Sigma-square ML     :       0.002                Log likelihood        :     123.044
S.E of regression   :       0.048                Akaike info criterion :    -230.088
                                                 Schwarz criterion     :    -211.338

------------------------------------------------------------------------------------
            Variable     Coefficient       Std.Error     z-Statistic     Probability
--------

/Users/polinarozhkova/anaconda3/lib/python3.10/site-packages/scipy/optimize/_minimize.py:892: RuntimeWarning: Method 'bounded' does not support relative tolerance in x; defaulting to absolute tolerance.
  warn("Method 'bounded' does not support relative tolerance in x; "


This final spatial lag model which excludes some of the explanatory variables from the previous models maintains estimated coefficient signs and the coefficients that were previously statistically signficant remain so. If I'm to compare between the two spatial lag models, the lower AIC here might be indicative of an improved fit. 

My plan is to try this again with different data for the dependent variable following the lead of most of the related literature which accounts for fatal and non-fatal overdoses. 